In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from guruTester import GuruTester
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates

# Write custom apply signal for strategy

In [2]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.low > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.high < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE        

In [3]:
pair = "USDCHF"
NB_H1_CANDLES = 2000

In [4]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
rates = Rates(pair, 13*NB_H1_CANDLES, mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD #= 0

9e-05

In [5]:
df_an = RSI(df_an)
df_an = apply_patterns(df_an)
df_an['EMA_200'] = df_an.close.ewm(span=200, min_periods=200).mean()
our_cols = ['time', 'open', 'high', 'low', 'close',
        'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [6]:
gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        SPREAD,
        use_spread=True
    )

prepare_data...


In [7]:
  df_res_m5 = gt.run_test()

run_test...


In [9]:
def calc_bernuli2(nb_tp,nb_sl,balance,profit_to_loss):
    loss_margin = 0.03
    return balance*( (1+profit_to_loss*loss_margin)**nb_tp )*( (1-loss_margin)**(nb_sl) )

In [ ]:
# def calc_balance(start,tp,sl,balance,loss_margin = 0.03):
def calc_balance()
    if(tp>0):
        return 
    if sl>0:
        return balance*(1-loss_margin)
        

In [14]:
df_res_m5

,running,start_index_m5,profit_factor,loss_factor,SPREAD,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,balance,time
0,False,4580,1.5,-1.0,0.00009,0.93739,0.93862,-1,0.935935,0.93836,-1.0,2022-12-05 00:05:00,2022-12-02 22:00:00,100.000000,2022-12-05 00:05:00
1,False,4734,1.5,-1.0,0.00009,0.93486,0.93688,-1,0.932535,0.93641,-1.0,2022-12-05 12:20:00,2022-12-05 11:00:00,97.000000,2022-12-05 12:20:00
2,False,4650,1.5,-1.0,0.00009,0.93555,0.93753,-1,0.932790,0.93739,-1.0,2022-12-05 16:15:00,2022-12-05 04:00:00,94.090000,2022-12-05 16:15:00
3,False,5428,1.5,-1.0,0.00009,0.93995,0.94052,-1,0.939275,0.94040,-1.0,2022-12-07 22:20:00,2022-12-07 21:00:00,91.267300,2022-12-07 22:20:00
4,False,5643,1.5,-1.0,0.00009,0.94012,0.93765,-1,0.937885,0.94161,1.5,2022-12-08 16:50:00,2022-12-08 15:00:00,95.374329,2022-12-08 16:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,False,25254,1.5,-1.0,0.00009,0.92954,0.92936,1,0.929765,0.92939,-1.0,2023-03-17 01:00:00,2023-03-17 01:00:00,66.691686,2023-03-17 01:00:00
139,False,25458,1.5,-1.0,0.00009,0.92854,0.92707,1,0.929920,0.92762,-1.0,2023-03-17 18:15:00,2023-03-17 18:00:00,64.690935,2023-03-17 18:15:00
140,False,25600,1.5,-1.0,0.00009,0.92593,0.92532,-1,0.925360,0.92631,1.5,2023-03-20 07:55:00,2023-03-20 07:00:00,67.602027,2023-03-20 07:55:00
141,False,25780,1.5,-1.0,0.00009,0.92871,0.92930,1,0.929190,0.92839,1.5,2023-03-20 22:25:00,2023-03-20 22:00:00,70.644118,2023-03-20 22:25:00


In [23]:
# df_res_m5['balance'] = 0

df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):# start from zero!!!!!!!!!!!!!!!!!!!!!
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = perv_balance*(1-loss_margin) 
    elif(df_res_m5.result[i] == 1.5):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]
    

C:\Users\meysam\AppData\Local\Temp\ipykernel_10108\924802483.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
from plotting import CandlePlot

In [25]:
df_res_m5['time'] = df_res_m5['end_time']
cp = CandlePlot(df_res_m5, candles=False)
cp.show_plot(line_traces=['balance'])